# CPSC 4300/6300-001 Applied Data Science (Fall 2020)

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Dane Acena"
COLLABORATORS = ""

# CPSC4300/6300-001 Problem Set #1

In this assignment, you will create a NCAA College Football team dataset by scraping data from the Internet and perform some basic data manipulations. The assignment has four parts: the first three are auto-graded. Th elast part is manually graded by the TA. 

Before submission, make sure you run all the cells. Because the TA could not run your notebook, your grade will be only based on the output and the writing part of your submitted notebook. 

## Part B: Scape the Rosters of Any NCAA Collge Football Team (50 points)

During the class, we have shown how to extract the roster of the Clemson Tiger's defense team, now we want to extend the work to apply to the entire team roster of any NCAA coolege football teams.

### Question 1. Wriet a function to computer the Team URL

Look at the URLs of server college football team at the ESPN site, generalize the pattern, and write a python function  `get_team_url(team_id, team_name)` to compute a url from team_id and team_name. (4 points)

In [2]:
def get_team_url(team_id, team_name):
    """
    Get the team URL for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        team_url: a string representing the URL of the team
    """
    url_team_name = team_name.replace(" ","-").lower()
    team_url = 'https://www.espn.com/college-football/team/roster/_/id/%s/%s' % (team_id,url_team_name)
    return team_url
#     raise NotImplementedError()

In [3]:
team_id = 228
team_name = "Clemson Tigers"
team_url = get_team_url(team_id, team_name)
assert team_url == 'https://www.espn.com/college-football/team/roster/_/id/228/clemson-tigers'

In [4]:
# This cell contains some hidden test cases.

### Question 2. Create a BeautifulSoup Object

Complete the following code to create a BeautifulSoup Object froma team url and name the object `soup`. (4 points)

In [5]:
url = get_team_url(228, "Clemson Tigers")

import requests
from bs4 import BeautifulSoup

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
# raise NotImplementedError()
type(soup)

bs4.BeautifulSoup

In [6]:
assert (type(soup) == BeautifulSoup)

In [7]:
assert soup.find('title').text.startswith('Clemson')

### Question 3. Find the Team Name

Write a statement to find a tag whose text is formatted as "{Team Name} Rosters" on the page. The {Team Name} would be the actual team name like 'Clemson Tigers'. (2 points)

In [8]:
team_tag = soup.find('h1', string='%s Roster' % team_name)
# raise NotImplementedError()
print(type(team_tag))
print(team_tag)

<class 'bs4.element.Tag'>
<h1 class="headline headline__h1 dib">Clemson Tigers Roster</h1>


In [9]:
assert str(type(team_tag)) == "<class 'bs4.element.Tag'>" and team_tag.text.endswith("Roster")

### Question 5. Search for Team Groups

A football team roster is usually divied into three groups: *Offense*, *Defense*, and *Special Teams*. Write some statements to find the tags for all these three groups and save the tags into a variable called groups. (2 points)

In [10]:
groups = soup.find_all(class_="Table__Title")
# raise NotImplementedError()
print(len(groups), "groups")
for g in groups:
    print(g)

3 groups
<div class="Table__Title">Offense</div>
<div class="Table__Title">Defense</div>
<div class="Table__Title">Special Teams</div>


In [11]:
assert ','.join(sorted([g.text for g in groups])) == 'Defense,Offense,Special Teams'

In [12]:
assert len(groups) == 3

### Question 6. Find the Column Names

Complete the code segment to extract the names of the columns in a table rows and save those names into a list named col_names. (4 points)

In [13]:
t = soup.find(class_="Table__Title", string="Offense").find_next("table")
col_names = [column.get_text() for column in t.find_all('th')]
# table_header = soup.find_all(class_="Table__TH")
# # raise NotImplementedError()
# for columns in table_header:
#     col_names.append(columns.text)

# table_header = soup.find(class_="Table__Title", string="Offense").find_next("table")
# for columns in t.thead.tr.children:
#     col_names.append(columns.text)

col_names

['', 'Name', 'POS', 'HT', 'WT', 'Class', 'Birthplace']

In [14]:
assert "Name" in col_names and "Class" in col_names

In [15]:
assert type(col_names) is list and len(col_names) >= 6

### Question 7. Find the Attributes of a Player

Write code segment to extract the attributes of the first defense player and save them into a dict. The player should have the attributes as shown in the following example.
```
player_name: Lannden Zanders
player_id: 4428971
player_no: 36
POS: S
HT: 6' 1"
WT: 195 lbs
Class: SO
Birthplace: Shelby, NC
```
(10 points)

In [16]:
import re

player = {}
players = []
player_attr = soup.find(class_="Table__Title", string="Defense").find_next('table')

def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False
    
for c in player_attr.tbody.find_all(is_player_row):
    tds = c.find_all('td')
    urls = c.find_all('a', href=True)
    cols = []   
    for td in tds:
        cols.append(td.get_text())
    for url in urls:
        if (url['href']) not in cols:
            cols.append(url['href'])
    players.append(cols)

player_name = "".join(re.split("[^a-zA-Z ]*", players[0][1]))
player_no = "".join(re.split("[^0-9]*", players[0][1]))

player_url = players[0][7]
player_id = "".join(re.split("[^0-9]*", player_url))
player = {
    "player_name": player_name,
    "player_id": player_id,
    "player_no": player_no,
    "POS": players[0][2],
    "HT": players[0][3],
    "WT": players[0][4],
    "Class": players[0][5],
    "Birthplace": players[0][6]
}
print(player)
# raise NotImplementedError()

{'player_name': 'Bryan Bresee', 'player_id': '4428988', 'player_no': '11', 'POS': 'DL', 'HT': '6\' 5"', 'WT': '300 lbs', 'Class': 'FR', 'Birthplace': 'Damascus, MD'}


In [17]:
for k, v in player.items():
    print("{}: {}".format(k, v))


player_name: Bryan Bresee
player_id: 4428988
player_no: 11
POS: DL
HT: 6' 5"
WT: 300 lbs
Class: FR
Birthplace: Damascus, MD


In [18]:
assert type(player) is dict

In [19]:
assert len(player) == 8

In [20]:
assert ','.join(sorted(player.keys())) == 'Birthplace,Class,HT,POS,WT,player_id,player_name,player_no'

In [21]:
assert int(player['player_no']) > 0 and int(player['player_no']) < 200

In [22]:
assert int(player['WT'].split()[0]) > 150 and player['WT'].split()[1] == 'lbs'

### Question 8. Abstract the tasks into a function

Since you don't want to repeat the above steps for every player and every team, it is necessary to abstract the above scraping task into a function which you can reuse over and over. (10 points)

In [23]:
import requests
from bs4 import BeautifulSoup

def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire team.
    """
    roster = []
    url_team_name = team_name.replace(" ","-").lower()
    team_url = 'https://www.espn.com/college-football/team/roster/_/id/%s/%s' % (team_id,url_team_name)
#     print(team_url)
    
    req = requests.get(team_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    t = soup.find(class_="Table__Title").find_next("table")
    col_names = [column.get_text() for column in t.find_all('th')]
    
    player = {}
    players = []
    i = 0
    player_groups = soup.find_all(class_="Table__Title")
    
    for group_roster in player_groups:
        player_group_attr = soup.find(class_="Table__Title",string=group_roster.text).find_next('table')
        for c in player_group_attr.tbody.find_all(is_player_row):
            
            tds = c.find_all('td')
            urls = c.find_all('a', href=True)
            cols = []   
            for td in tds:
                cols.append(td.get_text())
            for url in urls:
                if (url['href']) not in cols:
                    cols.append(url['href'])
            players.append(cols)
            
            
            player_name = "".join(re.split("[^a-zA-Z ]*", players[i][1]))
            player_no = "".join(re.split("[^0-9]*", players[i][1]))

            player_url = players[i][7]
            player_id = "".join(re.split("[^0-9]*", player_url))
            player = {
                "player_name": player_name,
                "player_id": player_id,
                "player_no": player_no,
                "POS": players[i][2],
                "HT": players[i][3],
                "WT": players[i][4],
                "Class": players[i][5],
                "Birthplace": players[i][6]
            }
            roster.append(player)
            i = i + 1
    
#     raise NotImplementedError()
#     print(roster)
    return roster

team_id = 150
team_name = "Duke Blue Devils"
team_roster = extract_team_roster(team_id, team_name)

In [24]:
# test the function
import pprint

team_id = 150
team_name = "Duke Blue Devils"
team_roster = extract_team_roster(team_id, team_name)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(team_roster[:3])

[   {   'Birthplace': 'Grayson, GA',
        'Class': 'JR',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '235 lbs',
        'player_id': '4239994',
        'player_name': 'Chase Brice',
        'player_no': '8'},
    {   'Birthplace': 'Langhorne, PA',
        'Class': 'SO',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '190 lbs',
        'player_id': '4568231',
        'player_name': 'Jack Colyar',
        'player_no': '13'},
    {   'Birthplace': 'Venice, CA',
        'Class': 'FR',
        'HT': '6\' 3"',
        'POS': 'QB',
        'WT': '180 lbs',
        'player_id': '4429578',
        'player_name': 'Luca Diamont',
        'player_no': '2'}]


In [25]:
type(team_roster) is list

True

In [26]:
assert len(team_roster) >= 80

In [27]:
assert ','.join(sorted(team_roster[50].keys())) == 'Birthplace,Class,HT,POS,WT,player_id,player_name,player_no'

In [28]:
import numpy as np
classes = np.array([p['Class'] for p in team_roster])
class_labels, class_counts = np.unique(classes, return_counts=True)
assert ''.join(class_labels.tolist()) == 'FRJRSOSR'

In [29]:
import numpy as np
player_ids = np.array([p['player_id'] for p in team_roster])
unqiue_ids = np.unique(player_ids)
assert len(unqiue_ids) == len(team_roster)

### Question 9. Debug and Testing Functions

When you write a program, you have to make sure that it always works correctly even if some exteral data (e.g., the ESPN website) may be corrupted. For example, Some players in a certain team may not have their player no. Your code should be able tp catch such exceptions and replace the missing player no with the value "NA". (10 points)

In [30]:
team_id = 258
team_name = "Virginia Cavaliers"
team_roster = extract_team_roster(team_id, team_name)
import numpy as np
positions = np.array([p['POS'] for p in team_roster])
position_label, position_counts = np.unique(positions, return_counts=True)
s = ''.join(position_label.tolist())
s

for dicts in team_roster:
    for key in dicts:
        if not dicts[key]:
            dicts[key] = "NA"
    print(dicts)
# raise NotImplementedError()

{'player_name': 'Ira Armstead', 'player_id': '4429219', 'player_no': '98', 'POS': 'QB', 'HT': '6\' 3"', 'WT': '200 lbs', 'Class': 'FR', 'Birthplace': 'South Bend, IN'}
{'player_name': 'Brennan Armstrong', 'player_id': '4370775', 'player_no': '5', 'POS': 'QB', 'HT': '6\' 2"', 'WT': '220 lbs', 'Class': 'SO', 'Birthplace': 'Shelby, OH'}
{'player_name': 'Jared Rayman', 'player_id': '4568497', 'player_no': 'NA', 'POS': 'QB', 'HT': '6\' 2"', 'WT': '180 lbs', 'Class': 'FR', 'Birthplace': 'Atlanta, GA'}
{'player_name': 'Lindell Stone', 'player_id': '4240850', 'player_no': '36', 'POS': 'QB', 'HT': '6\' 0"', 'WT': '230 lbs', 'Class': 'JR', 'Birthplace': 'Dallas, TX'}
{'player_name': 'Keytaon Thompson', 'player_id': '4242243', 'player_no': '99', 'POS': 'QB', 'HT': '6\' 4"', 'WT': '225 lbs', 'Class': 'JR', 'Birthplace': 'New Orleans, LA'}
{'player_name': 'Chris Barfield', 'player_id': '4684946', 'player_no': 'NA', 'POS': 'RB', 'HT': '5\' 10"', 'WT': '205 lbs', 'Class': 'FR', 'Birthplace': 'Richmon

In [31]:
import numpy as np
positions = np.array([p['POS'] for p in team_roster])
position_label, position_counts = np.unique(positions, return_counts=True)
assert ''.join(position_label.tolist()) == 'CCBDBDEDLDTFBGLBLSNTOLOTPPKQBRBSTEWR'

In [32]:
player_nos = np.array([p['player_no'] for p in team_roster])
na_count = len([no for no in player_nos if no == 'NA'])
assert na_count > 10

### Question 10. Add functions into a Python Modules

Now you have developed two solid functions: `get_team_url` and `extract_team_roster` and you would like to reuse them in other projects or share them with other people. Use the Notebook magic command to write the two functions with other neccessary  statements into a python file named `roster.py`. (4 points)

In [33]:
%%writefile roster.py
import re
import requests
from bs4 import BeautifulSoup
def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False

def get_team_url(team_id, team_name):
    """
    Get the team URL for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        team_url: a string representing the URL of the team
    """
    url_team_name = team_name.replace(" ","-").lower()
    team_url = 'https://www.espn.com/college-football/team/roster/_/id/%s/%s' % (team_id,url_team_name)
    return team_url

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire team.
    """
    roster = []
    url_team_name = team_name.replace(" ","-").lower()
    team_url = 'https://www.espn.com/college-football/team/roster/_/id/%s/%s' % (team_id,url_team_name)
    
    req = requests.get(team_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    t = soup.find(class_="Table__Title").find_next("table")
    col_names = [column.get_text() for column in t.find_all('th')]

    player = {}
    players = []
    i = 0
    player_groups = soup.find_all(class_="Table__Title")
    
    for group_roster in player_groups:
        player_group_attr = soup.find(class_="Table__Title",string=group_roster.text).find_next('table')
        for c in player_group_attr.tbody.find_all(is_player_row):
            
            tds = c.find_all('td')
            urls = c.find_all('a', href=True)
            cols = []   
            for td in tds:
                cols.append(td.get_text())
            for url in urls:
                if (url['href']) not in cols:
                    cols.append(url['href'])
            players.append(cols)
            
            
            player_name = "".join(re.split("[^a-zA-Z ]*", players[i][1]))
            player_no = "".join(re.split("[^0-9]*", players[i][1]))

            player_url = players[i][7]
            player_id = "".join(re.split("[^0-9]*", player_url))
            player = {
                "player_name": player_name,
                "player_id": player_id,
                "player_no": player_no,
                "POS": players[i][2],
                "HT": players[i][3],
                "WT": players[i][4],
                "Class": players[i][5],
                "Birthplace": players[i][6]
            }
            roster.append(player)
            i = i + 1

    return roster

Writing roster.py


In [34]:
!cat roster.py

import re
import requests
from bs4 import BeautifulSoup
def is_player_row(tag):
    if tag.name == 'tr' and tag.has_attr('data-idx'):
        return True
    return False

def get_team_url(team_id, team_name):
    """
    Get the team URL for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        team_url: a string representing the URL of the team
    """
    url_team_name = team_name.replace(" ","-").lower()
    team_url = 'https://www.espn.com/college-football/team/roster/_/id/%s/%s' % (team_id,url_team_name)
    return team_url

def extract_team_roster(team_id, team_name):
    """
    Extract the team roster for a NCAA college football team
    
    Args:
        team_id: the id of a NACC football team assigned by ESPN
        team_name: the name of the football team
        
    Returns:
        a list of dict representing the roster of the entire tea

In [35]:
import os
assert os.path.exists("roster.py")

In [36]:
try:
    from bs4 import BeautifulSoup
    import requests
    from roster import get_team_url, extract_team_roster
    
    clemson_tiger_roster = extract_team_roster(258, "Virginia Cavaliers")
except:
    raise AssertionError("could not import")


### Congratulations! Yo have finished Part B of the problem.